<a href="https://colab.research.google.com/github/kosirobwada/KnowledgeGraph/blob/main/PDCA1_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install wikipedia transformers

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=0d2d8934228818f8a3f1cf9aeeb9c5304010283fd2ababcba2898b26679421b2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [4]:
from transformers import BertModel, BertTokenizer
import wikipedia
import torch
from scipy.spatial.distance import cosine
import queue


In [5]:
def get_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()


In [6]:
# モデルとトークナイザーのロード
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [12]:
def bfs_search(start_page, target_word, model, tokenizer, max_depth=5):
    visited = set() # 既に訪れたページを追跡
    q = queue.Queue()
    q.put((start_page, [start_page])) # (現在のページ, パス)
    print(f"探索開始: {start_page} から {target_word} へのパスを探索します。")

    while not q.empty():
        current_page, path = q.get()
        if current_page in visited:
            continue
        visited.add(current_page)
        print(f"現在のページ: {current_page}, パス: {path}")

        # 現在のページのリンクを取得
        try:
            page_links = wikipedia.page(current_page).links
            print(f"取得したリンク数: {len(page_links)}")
        except:
            print(f"ページ {current_page} のリンク取得に失敗。")
            continue

        link_scores = []
        for link in page_links:
            try:
                link_embedding = get_embedding(link, model, tokenizer)
                target_embedding = get_embedding(target_word, model, tokenizer)
                similarity = 1 - cosine(link_embedding.detach().numpy(), target_embedding.detach().numpy())
                link_scores.append((link, similarity))
            except Exception as e:
                print(f"エンベディングまたは類似度計算中にエラー: {e}")
                continue

        # スコアに基づいてリンクをソート（降順）
        sorted_links = sorted(link_scores, key=lambda x: x[1], reverse=True)[:30] # 上位30個のリンク
        print(f"上位30個のリンクに追加")

        for link, score in sorted_links:
            if link == target_word:
                print(f"目的の単語 {target_word} に到達。パス: {path + [link]}")
                return path + [link] # 目的の単語に到達
            if len(path) < max_depth:
                q.put((link, path + [link])) # キューに追加
                print(f"キューに追加: {link}, 新しいパス: {path + [link]}")

    print("目的の単語に到達できなかった。")
    return None # 目的の単語に到達できなかった場合


In [13]:
# 実行時間を計測
import time
start_time = time.time()

# 幅優先探索の実行
start_page = "サッカー"
target_word = "サッカーボール"
path = bfs_search(start_page, target_word, model, tokenizer)

if path:
    print(f"Found path: {' -> '.join(path)}")
else:
    print("Path not found.")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

探索開始: サッカー から サッカーボール へのパスを探索します。
現在のページ: サッカー, パス: ['サッカー']
取得したリンク数: 101
上位30個のリンクに追加
キューに追加: FA Cup, 新しいパス: ['サッカー', 'FA Cup']
キューに追加: Association football, 新しいパス: ['サッカー', 'Association football']
キューに追加: EFL Cup, 新しいパス: ['サッカー', 'EFL Cup']
キューに追加: Japan national football team, 新しいパス: ['サッカー', 'Japan national football team']
キューに追加: Dan Potts (footballer), 新しいパス: ['サッカー', 'Dan Potts (footballer)']
キューに追加: Rob Edwards (footballer, born 1982), 新しいパス: ['サッカー', 'Rob Edwards (footballer, born 1982)']
キューに追加: EAFF E-1 Football Championship, 新しいパス: ['サッカー', 'EAFF E-1 Football Championship']
キューに追加: Jacob Brown (footballer), 新しいパス: ['サッカー', 'Jacob Brown (footballer)']
キューに追加: J.League, 新しいパス: ['サッカー', 'J.League']
キューに追加: Joe Taylor (footballer, born 2002), 新しいパス: ['サッカー', 'Joe Taylor (footballer, born 2002)']
キューに追加: AFC Champions League, 新しいパス: ['サッカー', 'AFC Champions League']
キューに追加: Japan national under-23 football team, 新しいパス: ['サッカー', 'Japan national under-23 football team']
キューに追加: J.

KeyboardInterrupt: 